In [1]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'

# Add the module Converters directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))



In [2]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

pd.options.mode.copy_on_write = True
engine = create_engine("sqlite:///../data/database.db", echo=False)

In [3]:
# -------------------
# CONFIGURABLE INPUTS
# -------------------

date_from = "2025-05-01"
date_through = "2025-09-30"

# Choose period granularity: "day", "week", "month", "quarter", "year"
period = "week"


In [4]:
# Mapping period to offset (end of period)
period_offsets = {
    "day": pd.offsets.Day(0),
    "week": pd.offsets.Week(weekday=6),         # Sunday
    "month": pd.offsets.MonthEnd(0),
    "quarter": pd.offsets.QuarterEnd(startingMonth=12),
    "year": pd.offsets.YearEnd(0)
}
date_offset = period_offsets.get(period, pd.offsets.MonthEnd(0))
date_freq = {
    "day": "D",
    "week": "W-SUN",         # Sunday
    "month": "ME",
    "quarter": "QE",
    "year": "YE"
}
date_frequency = date_freq.get(period, "ME")

In [5]:
# Load all data from database

# handling sqlite issues with datetime timepart
next_day_date_through = (pd.to_datetime(date_through) + pd.Timedelta(days=1)).strftime('%Y-%m-%d')
previous_day_date_from = (pd.to_datetime(date_from) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

actual = pd.read_sql_query('SELECT * FROM G09_CashFlow_Actual_Corresponding WHERE date < "' + next_day_date_through + '" ', engine)
pending = pd.read_sql_query('SELECT * FROM G12_CashFlow_Pending_Corresponding WHERE p_date < "' + next_day_date_through + '" ', engine)
budgeted = pd.read_sql_query('SELECT * FROM F01_BudgetEntries WHERE date > "' + previous_day_date_from + '" AND date < "' + next_day_date_through + '" ', engine)
cash = pd.read_sql_query('SELECT * FROM G01_CashTransactions WHERE date < "' + next_day_date_through + '" ', engine)
definition_df = pd.read_sql_table("E01_CashFlowDefinition", engine)

definition_accounts_df = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
definition_totals_df = pd.read_sql_table("E01_CashFlowDefinitionTotals", engine)



In [6]:
# Ensure required columns and formats

actual['period_end'] = pd.to_datetime(actual['date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
actual['cf_amount'] = np.where(actual['entry_type'] == 'CR', actual['amount_LC'], -actual['amount_LC'])
pending['period_end'] = pd.to_datetime(pending['p_date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
pending['cf_amount'] = np.where(pending['entry_type'] == 'CR', pending['p_amount_LC'], -pending['p_amount_LC'])
budgeted['period_end'] = pd.to_datetime(budgeted['date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
budgeted['cf_amount'] = np.where(budgeted['cash_type'] == 'Receipt', budgeted['amount_LC'], -budgeted['amount_LC'])
cash['period_end'] = pd.to_datetime(cash['date'], format='mixed', errors='coerce').apply(lambda d: date_offset.rollforward(d) if pd.notnull(d) else pd.NaT)
cash['cf_amount'] = np.where(cash['entry_type'] == 'DR', cash['amount_LC'], -cash['amount_LC'])

definition_df.rename(columns={"id":"definition_id"}, inplace=True)
definition_acc_df = definition_df[definition_df["definition_type"] == 1]
definition_tot_df = definition_df[definition_df["definition_type"] == 2]
definition_bal_df = definition_df[definition_df["definition_type"] == 3]

In [7]:
actual.head()

,d_id,cash_status,cash_type,d_type,date,number,partner_id,partner_name,description,currency,entry_type,account,account_type,amount,amount_LC,period_end,cf_amount
0,56036,Actual,Payment,Izej.b.dok.,2025-04-12 00:00:00.000000,None,13239,"NADĪNE, SIA",None,EUR,DR,5721,NaN,67.69,67.69,2025-04-13,-67.69
1,56036,Actual,Payment,Izej.b.dok.,2025-04-12 00:00:00.000000,None,13239,"NADĪNE, SIA",None,EUR,DR,7170,NaN,322.31,322.31,2025-04-13,-322.31
2,56013,Actual,Receipt,Ienāk.b.dok.,2025-05-12 00:00:00.000000,mi-2,13235,"LEDIŅI, SIA",None,EUR,CR,2310,2.0,33.20,33.20,2025-05-18,33.20
3,56013,Actual,Receipt,Ienāk.b.dok.,2025-05-12 00:00:00.000000,mi-2,13235,"LEDIŅI, SIA",None,EUR,CR,5721,NaN,16.80,16.80,2025-05-18,16.80
4,56013,Actual,Receipt,Ienāk.b.dok.,2025-05-12 00:00:00.000000,mi-2,13235,"LEDIŅI, SIA",None,EUR,CR,6110,NaN,80.00,80.00,2025-05-18,80.00


In [8]:
# actual.to_excel("actual_june.xlsx")

In [9]:
pending.head()

,d_id,cash_status,cash_type,number,date,p_date,description,partner_id,partner_name,currency,entry_type,account,account_type,p_amount,p_amount_LC,period_end,cf_amount
0,56013,Pending,Payment,mi-2,2025-05-12 00:00:00.000000,2025-09-14,None,13235,"LEDIŅI, SIA",EUR,DR,2620,1.0,33.2,33.2,2025-09-14,-33.2
1,56016,Pending,Payment,MU-1,2025-06-30 00:00:00.000000,2025-09-14,Apmaksa par realizāciju,1,Noliktava (galvenā),EUR,DR,2620,1.0,250.0,250.0,2025-09-14,-250.0
2,56017,Pending,Receipt,MI-1,2025-06-30 00:00:00.000000,2025-09-14,None,13237,"BITE, Sabiedrība ar ierobežotu atbildību",EUR,CR,2620,1.0,50.0,50.0,2025-09-14,50.0
3,56023,Pending,Payment,0001,2025-06-28 00:00:00.000000,2025-09-14,None,13235,"LEDIŅI, SIA",EUR,DR,5721,NaN,10.5,10.5,2025-09-14,-10.5
4,56023,Pending,Payment,0001,2025-06-28 00:00:00.000000,2025-09-14,None,13235,"LEDIŅI, SIA",EUR,DR,7170,NaN,50.0,50.0,2025-09-14,-50.0


In [10]:
# pending.to_excel("pending_june.xlsx")

In [11]:
budgeted.head()

,id,definition_id,cash_type,date,amount_LC,memo,period_end,cf_amount


In [12]:
# -------------------------------------------------------------------
# ************** (1) CashFlow based on accounts definitions *********
# -------------------------------------------------------------------

#  Prepare chart of CF definitions 

# Merge accounts into accounts definition
definition_accounts_df = pd.merge(
    definition_acc_df,
    definition_accounts_df,
    on="definition_id", how="left")
definition_accounts_df.drop(columns=["id"], inplace=True)

In [13]:
definition_accounts_df

,definition_id,key,definition_type,name,cash_type,account
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110
1,3,102,1.0,Maksājumi piegādātājiem,Payment,7110
2,4,103,1.0,Maksājumi darbiniekiem,Payment,7210
3,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,6550
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,2310
5,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,5310
6,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,53
7,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,11
8,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,7170
9,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,2620


In [14]:
actual = pd.merge(definition_accounts_df,
    actual,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'account'], # Corresponding columns in transactions_df
    how='left'
)


In [15]:
actual.head()

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,d_type,date,...,partner_id,partner_name,description,currency,entry_type,account_type,amount,amount_LC,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,56013.0,Actual,Ienāk.b.dok.,2025-05-12 00:00:00.000000,...,13235.0,"LEDIŅI, SIA",None,EUR,CR,NaN,80.0,80.0,2025-05-18,80.0
1,3,102,1.0,Maksājumi piegādātājiem,Payment,7110,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,4,103,1.0,Maksājumi darbiniekiem,Payment,7210,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,6550,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,2310,56013.0,Actual,Ienāk.b.dok.,2025-05-12 00:00:00.000000,...,13235.0,"LEDIŅI, SIA",None,EUR,CR,2.0,33.2,33.2,2025-05-18,33.2


In [16]:
pending = pd.merge(definition_accounts_df,
    pending,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'account'], # Corresponding columns in transactions_df
    how='left'
)


In [17]:
pending.head(10)

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,number,date,...,description,partner_id,partner_name,currency,entry_type,account_type,p_amount,p_amount_LC,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,56032.0,Pending,0001,2025-03-10 00:00:00.000000,...,None,13239.0,"NADĪNE, SIA",USD,CR,NaN,350.00000,322.37000,2025-09-14,322.37000
1,3,102,1.0,Maksājumi piegādātājiem,Payment,7110,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,4,103,1.0,Maksājumi darbiniekiem,Payment,7210,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,6550,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,2310,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,5310,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,53,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,7170,56023.0,Pending,0001,2025-06-28 00:00:00.000000,...,None,13235.0,"LEDIŅI, SIA",EUR,DR,NaN,50.00000,50.00000,2025-09-14,-50.00000
9,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,7170,56033.0,Pending,0002,2025-07-01 00:00:00.000000,...,None,13239.0,"NADĪNE, SIA",EUR,DR,NaN,927.68595,927.68595,2025-09-14,-927.68595


In [18]:
budgeted = pd.merge(definition_acc_df,
    budgeted,
    left_on=['definition_id'],        # Columns in definition_df
    right_on=['definition_id'], # Corresponding columns in transactions_df
    how='left'
)


In [19]:
budgeted.head()

,definition_id,key,definition_type,name,id,cash_type,date,amount_LC,memo,period_end,cf_amount
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,3,102,1.0,Maksājumi piegādātājiem,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,4,103,1.0,Maksājumi darbiniekiem,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,7,201,1.0,Izdevumi procentu maksājumiem,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [20]:
# -------------------
# GROUP & MERGE
# -------------------
def get_period_totals(df, label):
    grouped = df.groupby(['definition_id', 'period_end', 'cash_type'])['cf_amount'].sum().unstack(fill_value=0)
    columns = ["Receipt", "Payment"]
    grouped = grouped.reindex(columns=columns, fill_value=0)
    grouped.columns = [f"{label}_{col}" for col in grouped.columns]
    return grouped

In [21]:
actual_period = get_period_totals(actual, 'Actual')
pending_period = get_period_totals(pending, 'Pending')
budgeted_period = get_period_totals(budgeted, 'Budgeted')

In [22]:
actual_period

Actual_Receipt  Actual_Payment
definition_id period_end                                
2             2025-05-18            80.0            0.00
5             2025-04-13             0.0         -322.31
              2025-05-18            33.2            0.00
              2025-07-06           250.0          -50.00
              2025-07-13             0.0         -210.00
8             2025-04-13             0.0          -67.69
              2025-05-18            16.8            0.00

In [23]:
pending_period

,,Pending_Receipt,Pending_Payment
definition_id,period_end,,
2,2025-09-14,322.37,0.00000
5,2025-09-14,50.00,-1260.88595
8,2025-09-14,67.70,-205.31405


In [24]:
# Combine all
combined = actual_period.join(pending_period, how='outer') \
                        .join(budgeted_period, how='outer') \
                        .fillna(0).reset_index()

In [25]:
combined.head(10)

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment
0,2,2025-05-18,80.0,0.00,0.00,0.00000,0.0,0.0
1,2,2025-09-14,0.0,0.00,322.37,0.00000,0.0,0.0
2,5,2025-04-13,0.0,-322.31,0.00,0.00000,0.0,0.0
3,5,2025-05-18,33.2,0.00,0.00,0.00000,0.0,0.0
4,5,2025-07-06,250.0,-50.00,0.00,0.00000,0.0,0.0
5,5,2025-07-13,0.0,-210.00,0.00,0.00000,0.0,0.0
6,5,2025-09-14,0.0,0.00,50.00,-1260.88595,0.0,0.0
7,8,2025-04-13,0.0,-67.69,0.00,0.00000,0.0,0.0
8,8,2025-05-18,16.8,0.00,0.00,0.00000,0.0,0.0
9,8,2025-09-14,0.0,0.00,67.70,-205.31405,0.0,0.0


In [207]:
# combined.to_excel("Combined_june_day.xlsx")

In [208]:
# -------------------
# SPLIT PAST / FUTURE
# -------------------
today = pd.to_datetime(datetime.date.today())
this_period_end = date_offset.rollforward(today)

# Split
past = combined[combined['period_end'] < this_period_end]
future = combined[combined['period_end'] >= this_period_end]


In [209]:
# -------------------
# CASHFLOW LOGIC
# -------------------

# For past: use only actuals
past['income'] = past.get('Actual_Receipt', 0)
past['expense'] = past.get('Actual_Payment', 0)

In [216]:
past.tail()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,income,expense
3,5,2025-05-18,33.2,0.00,0.0,0.0,0.0,0.0,33.2,0.00
4,5,2025-07-06,250.0,-50.00,0.0,0.0,0.0,0.0,250.0,-50.00
5,5,2025-07-13,0.0,-210.00,0.0,0.0,0.0,0.0,0.0,-210.00
7,8,2025-04-13,0.0,-67.69,0.0,0.0,0.0,0.0,0.0,-67.69
8,8,2025-05-18,16.8,0.00,0.0,0.0,0.0,0.0,16.8,0.00


In [217]:
# past.to_excel("June_past_day.xlsx")

In [218]:
# For future: use max(budgeted, actual+pending)
future['actual_plus_pending_income'] = future.get('Actual_Receipt', 0) + future.get('Pending_Receipt', 0)
future['actual_plus_pending_expense'] = future.get('Actual_Payment', 0) + future.get('Pending_Payment', 0)

future['income'] = future[['Budgeted_Receipt', 'actual_plus_pending_income']].max(axis=1)
future['expense'] = future[['Budgeted_Payment', 'actual_plus_pending_expense']].min(axis=1)



In [219]:
future.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,actual_plus_pending_income,actual_plus_pending_expense,income,expense
1,2,2025-09-14,0.0,0.0,322.37,0.00000,0.0,0.0,322.37,0.00000,322.37,0.00000
6,5,2025-09-14,0.0,0.0,50.00,-1260.88595,0.0,0.0,50.00,-1260.88595,50.00,-1260.88595
9,8,2025-09-14,0.0,0.0,67.70,-205.31405,0.0,0.0,67.70,-205.31405,67.70,-205.31405


In [220]:
# future.to_excel("Future_june_day.xlsx")

In [221]:
# Combine
cashflow = pd.concat([past, future], ignore_index=True)
cashflow['net_cashflow'] = cashflow['income'] + cashflow['expense']

In [222]:
cashflow.head()

,definition_id,period_end,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,income,expense,actual_plus_pending_income,actual_plus_pending_expense,net_cashflow
0,2,2025-05-18,80.0,0.00,0.0,0.0,0.0,0.0,80.0,0.00,NaN,NaN,80.00
1,5,2025-04-13,0.0,-322.31,0.0,0.0,0.0,0.0,0.0,-322.31,NaN,NaN,-322.31
2,5,2025-05-18,33.2,0.00,0.0,0.0,0.0,0.0,33.2,0.00,NaN,NaN,33.20
3,5,2025-07-06,250.0,-50.00,0.0,0.0,0.0,0.0,250.0,-50.00,NaN,NaN,200.00
4,5,2025-07-13,0.0,-210.00,0.0,0.0,0.0,0.0,0.0,-210.00,NaN,NaN,-210.00


In [223]:
# Create and store cashflow checking report

checking_report = pd.merge(definition_acc_df, cashflow, on="definition_id", how="right")
checking_report.sort_values(["period_end", "key"], inplace=True)
checking_report.drop(columns=["key", "definition_type"], inplace=True) 
checking_report = checking_report.reindex(columns=["period_end", "definition_id", "name", "Actual_Receipt", "Actual_Payment", "Pending_Receipt", "Pending_Payment", 
                                 "Budgeted_Receipt", "Budgeted_Payment", "actual_plus_pending_income", "actual_plus_pending_expense",
                                 "income", "expense", "net_cashflow"])
checking_report = checking_report[checking_report["period_end"] >= pd.to_datetime(date_from)]


In [224]:
checking_report.head()

,period_end,definition_id,name,Actual_Receipt,Actual_Payment,Pending_Receipt,Pending_Payment,Budgeted_Receipt,Budgeted_Payment,actual_plus_pending_income,actual_plus_pending_expense,income,expense,net_cashflow
0,2025-05-18,2,Ieņēmumi no preču un pakalpojumu pārdošanas,80.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,80.0,0.0,80.0
2,2025-05-18,5,Pārējie pamatdarbības ieņēmumi un izdevumi,33.2,0.0,0.0,0.0,0.0,0.0,NaN,NaN,33.2,0.0,33.2
6,2025-05-18,8,Izdevumi nodokļu maksājumiem,16.8,0.0,0.0,0.0,0.0,0.0,NaN,NaN,16.8,0.0,16.8
3,2025-07-06,5,Pārējie pamatdarbības ieņēmumi un izdevumi,250.0,-50.0,0.0,0.0,0.0,0.0,NaN,NaN,250.0,-50.0,200.0
4,2025-07-13,5,Pārējie pamatdarbības ieņēmumi un izdevumi,0.0,-210.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,-210.0,-210.0


In [225]:
# -------------------
# CASHFLOW OUTPUT
# -------------------

# Pivot first
pivot_all_periods = cashflow.pivot_table(
    index='definition_id',
    columns='period_end',
    values='net_cashflow',
    aggfunc='sum'
).fillna(0)


In [226]:
pivot_all_periods

period_end,2025-04-13,2025-05-18,2025-07-06,2025-07-13,2025-09-14
definition_id,,,,,
2,0.00,80.0,0.0,0.0,322.37000
5,-322.31,33.2,200.0,-210.0,-1210.88595
8,-67.69,16.8,0.0,0.0,-137.61405


In [227]:
# Fill missing periods and drop unnecessary

# Build full period range
min_period = date_offset.rollforward(pd.to_datetime(date_from))
max_period = date_offset.rollforward(pd.to_datetime(date_through))

report_periods = pd.date_range(start=min_period, end=max_period, freq=date_frequency)

# Reindex pivot to include all requested periods
pivot_cf = pivot_all_periods.reindex(columns=report_periods, fill_value=0)

# Sort columns just in case
pivot_cf = pivot_cf.sort_index(axis=1)

In [228]:
pivot_cf

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,80.0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,322.37000,0,0,0
5,0,0,33.2,0,0,0,0,0,0,200.0,...,0,0,0,0,0,0,-1210.88595,0,0,0
8,0,0,16.8,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,-137.61405,0,0,0


In [229]:
# Store pivot_cf for cash flow figure
graph_pivot = pd.merge(definition_acc_df, pivot_cf, on="definition_id", how="right")
graph_pivot.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
graph_pivot=graph_pivot.set_index("name")

In [230]:
graph_pivot

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
name,,,,,,,,,,,,,,,,,,,,,
Ieņēmumi no preču un pakalpojumu pārdošanas,0,0,80.0,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,322.37000,0,0,0
Pārējie pamatdarbības ieņēmumi un izdevumi,0,0,33.2,0,0,0,0,0,0,200.0,...,0,0,0,0,0,0,-1210.88595,0,0,0
Izdevumi nodokļu maksājumiem,0,0,16.8,0,0,0,0,0,0,0.0,...,0,0,0,0,0,0,-137.61405,0,0,0


In [231]:
# Ensure 0 instead of NaN in empty rows
pivot_cf = pd.merge(definition_acc_df["definition_id"], pivot_cf, left_on="definition_id", right_on="definition_id", how="left").fillna(0)
pivot_cf.set_index("definition_id", inplace=True)

In [232]:
pivot_cf

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
5,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
8,0.0,0.0,16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-137.61405,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [233]:
# -----------------------------------------------------------------------------------------------------------------
# ********************** (2) Start working with subtotals based on totaled definitions *******************************
# -----------------------------------------------------------------------------------------------------------------

# Merge totals into totals definition
definition_totals_df = pd.merge(
    definition_tot_df,
    definition_totals_df,
    on="definition_id", how="left")
definition_totals_df.drop(columns=["id"], inplace=True)


In [234]:
definition_totals_df

,definition_id,key,definition_type,name,definition_summarized
0,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,2
1,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,3
2,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,4
3,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,5
4,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,2
5,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,3
6,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,4
7,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,5
8,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,7
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,8


In [235]:
# Merge totals definition with summarized accounts pivot

merged_totals = pd.merge(definition_totals_df,
    pivot_cf,
    left_on='definition_summarized',
    right_on='definition_id', 
    how='left'
)

# Drop unnecessary columns

merged_totals.drop(columns = ['key', 'definition_type', 'name', 'definition_summarized'], inplace=True)


In [236]:
merged_totals

,definition_id,2025-05-04 00:00:00,2025-05-11 00:00:00,2025-05-18 00:00:00,2025-05-25 00:00:00,2025-06-01 00:00:00,2025-06-08 00:00:00,2025-06-15 00:00:00,2025-06-22 00:00:00,2025-06-29 00:00:00,...,2025-08-03 00:00:00,2025-08-10 00:00:00,2025-08-17 00:00:00,2025-08-24 00:00:00,2025-08-31 00:00:00,2025-09-07 00:00:00,2025-09-14 00:00:00,2025-09-21 00:00:00,2025-09-28 00:00:00,2025-10-05 00:00:00
0,6,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.0,0.0,0.0
1,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
3,6,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.0,0.0,0.0
4,10,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.0,0.0,0.0
5,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
6,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
7,10,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.0,0.0,0.0
8,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
9,10,0.0,0.0,16.8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-137.61405,0.0,0.0,0.0


In [237]:
# Choose value_columns for further summarization

value_columns = [col for col in merged_totals.columns if col not in ['definition_id']]

In [238]:
value_columns

[Timestamp('2025-05-04 00:00:00'),
 Timestamp('2025-05-11 00:00:00'),
 Timestamp('2025-05-18 00:00:00'),
 Timestamp('2025-05-25 00:00:00'),
 Timestamp('2025-06-01 00:00:00'),
 Timestamp('2025-06-08 00:00:00'),
 Timestamp('2025-06-15 00:00:00'),
 Timestamp('2025-06-22 00:00:00'),
 Timestamp('2025-06-29 00:00:00'),
 Timestamp('2025-07-06 00:00:00'),
 Timestamp('2025-07-13 00:00:00'),
 Timestamp('2025-07-20 00:00:00'),
 Timestamp('2025-07-27 00:00:00'),
 Timestamp('2025-08-03 00:00:00'),
 Timestamp('2025-08-10 00:00:00'),
 Timestamp('2025-08-17 00:00:00'),
 Timestamp('2025-08-24 00:00:00'),
 Timestamp('2025-08-31 00:00:00'),
 Timestamp('2025-09-07 00:00:00'),
 Timestamp('2025-09-14 00:00:00'),
 Timestamp('2025-09-21 00:00:00'),
 Timestamp('2025-09-28 00:00:00'),
 Timestamp('2025-10-05 00:00:00')]

In [239]:
# Summarize value_columns based on group value

summarized_totals = merged_totals.groupby('definition_id', group_keys=False)[value_columns].sum()

# .reset_index()

summarized_totals.fillna(0, inplace=True)

In [240]:
summarized_totals

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,113.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-888.51595,0.0,0.0,0.0
10,0.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1026.13000,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
29,0.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1026.13000,0.0,0.0,0.0


In [241]:
# ------------------------------------------------------------------------------------------------------------
# ********************** (3) Start working with balances on end of each period *******************************
# ------------------------------------------------------------------------------------------------------------



In [242]:
# Step 1. Determine periods
today = pd.to_datetime(datetime.date.today())
this_period_end = date_offset.rollforward(today)
min_period = date_offset.rollforward(pd.to_datetime(date_from))
max_period = date_offset.rollforward(pd.to_datetime(date_through))

report_periods = pd.date_range(start=min_period, end=max_period, freq=date_frequency)

In [243]:
# Step 2. Determine past balances

# Group and sort actual bank balance by period
actual_cash_by_period = cash.groupby('period_end')['cf_amount'].sum().sort_index()
actual_cumulative_cash = actual_cash_by_period.cumsum()

In [244]:
actual_cumulative_cash

period_end
2025-04-13    -390
2025-05-18    -260
2025-07-06     -60
2025-07-13    -270
2025-07-20   -1270
Name: cf_amount, dtype: int64

In [245]:
if max_period >= this_period_end:
    past_balances = actual_cumulative_cash[actual_cumulative_cash.index < this_period_end]
else:
    past_balances = actual_cumulative_cash[actual_cumulative_cash.index <= max_period]



In [246]:
past_balances

period_end
2025-04-13    -390
2025-05-18    -260
2025-07-06     -60
2025-07-13    -270
2025-07-20   -1270
Name: cf_amount, dtype: int64

In [247]:
# Step 3. Determine future balances

if min_period >= this_period_end:
    future_periods = pd.date_range(start=this_period_end, end=max_period, freq=date_frequency)
else:
    future_periods = report_periods[report_periods >= this_period_end]

In [248]:
future_periods

DatetimeIndex(['2025-09-14', '2025-09-21', '2025-09-28', '2025-10-05'], dtype='datetime64[ns]', freq='W-SUN')

In [249]:
last_actual_balance = past_balances.iloc[-1] if not past_balances.empty else 0

In [250]:
last_actual_balance

-1270

In [251]:
# Future cashflows from pivot_all_periods
cashflow_by_period = pivot_all_periods.sum(axis=0)

bal = last_actual_balance
future_balances = {}
for period in future_periods:
    bal += cashflow_by_period.get(period, 0)
    future_balances[period] = bal



In [252]:
future_balances

{Timestamp('2025-09-14 00:00:00'): -2296.13,
 Timestamp('2025-09-21 00:00:00'): -2296.13,
 Timestamp('2025-09-28 00:00:00'): -2296.13,
 Timestamp('2025-10-05 00:00:00'): -2296.13}

In [253]:
# Step 4. Combine past and future into all closing balance series, adjust periods
all_closing_balances = pd.Series(dtype=float)
all_closing_balances = pd.concat([
    past_balances,
    pd.Series(future_balances)
]).reindex(report_periods, method="ffill")


In [254]:
all_closing_balances

2025-05-04    -390.00
2025-05-11    -390.00
2025-05-18    -260.00
2025-05-25    -260.00
2025-06-01    -260.00
2025-06-08    -260.00
2025-06-15    -260.00
2025-06-22    -260.00
2025-06-29    -260.00
2025-07-06     -60.00
2025-07-13    -270.00
2025-07-20   -1270.00
2025-07-27   -1270.00
2025-08-03   -1270.00
2025-08-10   -1270.00
2025-08-17   -1270.00
2025-08-24   -1270.00
2025-08-31   -1270.00
2025-09-07   -1270.00
2025-09-14   -2296.13
2025-09-21   -2296.13
2025-09-28   -2296.13
2025-10-05   -2296.13
Freq: W-SUN, dtype: float64

In [255]:
# Step 5. Formating

# Create a new DataFrame with closing balances only
balances = pd.DataFrame(
    [all_closing_balances],  # one row
    index=[0]  # index = 0 
)

In [256]:
# Store balances for use in Graph
graph_balances = all_closing_balances.values

In [257]:
graph_balances

array([ -390.  ,  -390.  ,  -260.  ,  -260.  ,  -260.  ,  -260.  ,
        -260.  ,  -260.  ,  -260.  ,   -60.  ,  -270.  , -1270.  ,
       -1270.  , -1270.  , -1270.  , -1270.  , -1270.  , -1270.  ,
       -1270.  , -2296.13, -2296.13, -2296.13, -2296.13])

In [258]:


# Create a new DataFrame with repeated rows for each definition_id
balances = pd.merge(definition_bal_df["definition_id"], balances, how='cross')
balances = balances.set_index("definition_id")


In [259]:
balances

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
30,-390.0,-390.0,-260.0,-260.0,-260.0,-260.0,-260.0,-260.0,-260.0,-60.0,...,-1270.0,-1270.0,-1270.0,-1270.0,-1270.0,-1270.0,-2296.13,-2296.13,-2296.13,-2296.13


In [260]:
# ---------------------------------------------------------------------------------------------------------------------------------------
# ********************** (4) Put together summarized accounts, totals and balances on end of each period *******************************
# ---------------------------------------------------------------------------------------------------------------------------------------



In [261]:
pivot_cf

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
5,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
8,0.0,0.0,16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-137.61405,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0


In [262]:
summarized_totals

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
6,0.0,0.0,113.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-888.51595,0.0,0.0,0.0
10,0.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1026.13000,0.0,0.0,0.0
20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
29,0.0,0.0,130.0,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1026.13000,0.0,0.0,0.0


In [263]:
balances

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
30,-390.0,-390.0,-260.0,-260.0,-260.0,-260.0,-260.0,-260.0,-260.0,-60.0,...,-1270.0,-1270.0,-1270.0,-1270.0,-1270.0,-1270.0,-2296.13,-2296.13,-2296.13,-2296.13


In [264]:
# Concatenate it all together
report = pd.concat([pivot_cf, summarized_totals, balances])

In [265]:
report

,2025-05-04,2025-05-11,2025-05-18,2025-05-25,2025-06-01,2025-06-08,2025-06-15,2025-06-22,2025-06-29,2025-07-06,...,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05
definition_id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.00,0.00,0.00
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
5,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.00,0.00,0.00
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
8,0.0,0.0,16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-137.61405,0.00,0.00,0.00
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00


In [266]:
# -------------------------------------
# Prepare report for visual appearance 
# -------------------------------------

# Merge report definition header with report
report = pd.merge(definition_df, report, left_on="definition_id", right_on="definition_id", how="left")

# Sort based on key value
report.sort_values("key", inplace=True)


In [267]:
report

,definition_id,key,definition_type,name,2025-05-04 00:00:00,2025-05-11 00:00:00,2025-05-18 00:00:00,2025-05-25 00:00:00,2025-06-01 00:00:00,2025-06-08 00:00:00,...,2025-08-03 00:00:00,2025-08-10 00:00:00,2025-08-17 00:00:00,2025-08-24 00:00:00,2025-08-31 00:00:00,2025-09-07 00:00:00,2025-09-14 00:00:00,2025-09-21 00:00:00,2025-09-28 00:00:00,2025-10-05 00:00:00
0,1,100,NaN,PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,0.0,0.0,80.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.00,0.00,0.00
2,3,102,1.0,Maksājumi piegādātājiem,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
3,4,103,1.0,Maksājumi darbiniekiem,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,0.0,0.0,33.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.00,0.00,0.00
5,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,0.0,0.0,113.2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-888.51595,0.00,0.00,0.00
6,7,201,1.0,Izdevumi procentu maksājumiem,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
7,8,202,1.0,Izdevumi nodokļu maksājumiem,0.0,0.0,16.8,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-137.61405,0.00,0.00,0.00
8,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,0.0,0.0,130.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1026.13000,0.00,0.00,0.00


In [268]:
# Subtract report formatting in separate dataframe
format_df = report["definition_type"]

In [269]:
format_df

0     NaN
1     1.0
2     1.0
3     1.0
4     1.0
5     2.0
6     1.0
7     1.0
8     1.0
9     2.0
10    NaN
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    2.0
20    NaN
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    2.0
27    1.0
28    2.0
29    3.0
Name: definition_type, dtype: float64

In [270]:
# Prepare report_df for visual appearance

report.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
report.set_index("name", inplace=True)

# Format column headers to show only the date part
report.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in report.columns]

In [271]:
report

,"04. May, 2025","11. May, 2025","18. May, 2025","25. May, 2025","01. Jun, 2025","08. Jun, 2025","15. Jun, 2025","22. Jun, 2025","29. Jun, 2025","06. Jul, 2025",...,"03. Aug, 2025","10. Aug, 2025","17. Aug, 2025","24. Aug, 2025","31. Aug, 2025","07. Sep, 2025","14. Sep, 2025","21. Sep, 2025","28. Sep, 2025","05. Oct, 2025"
name,,,,,,,,,,,,,,,,,,,,,
PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ieņēmumi no preču un pakalpojumu pārdošanas,0.0,0.0,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,322.37000,0.00,0.00,0.00
Maksājumi piegādātājiem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
Maksājumi darbiniekiem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
Pārējie pamatdarbības ieņēmumi un izdevumi,0.0,0.0,33.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-1210.88595,0.00,0.00,0.00
BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,0.0,0.0,113.2,0.0,0.0,0.0,0.0,0.0,0.0,200.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-888.51595,0.00,0.00,0.00
Izdevumi procentu maksājumiem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
Izdevumi nodokļu maksājumiem,0.0,0.0,16.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-137.61405,0.00,0.00,0.00
Naudas plūsma no ārkārtas posteņiem,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.00,0.00,0.00
